In [1]:
# import imp library
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import numpy as np


In [2]:
faqs = """About the Program
What is the course fee for  Data Science Mentorship Program (DSMP 2023)
The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.
What is the total duration of the course?
The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)
What is the syllabus of the mentorship program?
We will be covering the following modules:
Python Fundamentals
Python libraries for Data Science
Data Analysis
SQL for Data Science
Maths for Machine Learning
ML Algorithms
Practical ML
MLOPs
Case studies
Will Deep Learning and NLP be a part of this program?
No, NLP and Deep Learning both are not a part of this program’s curriculum.
What if I miss a live session? Will I get a recording of the session?
Yes all our sessions are recorded, so even if you miss a session you can go back and watch the recording.
Where can I find the class schedule?
What is the time duration of all the live sessions?
Roughly, all the sessions last 2 hours.
What is the language spoken by the instructor during the sessions?
Hinglish
How will I be informed about the upcoming class?
You will get a mail from our side before every paid session once you become a paid user.
Can I do this course if I am from a non-tech background?
Yes, absolutely.
I am late, can I join the program in the middle?
Absolutely, you can join the program anytime.
If I join/pay in the middle, will I be able to see all the past lectures?
Yes, once you make the payment you will be able to see all the past content in your dashboard.
Where do I have to submit the task?
You don’t have to submit the task. We will provide you with the solutions, you have to self evaluate the task yourself.
Will we do case studies in the program?
Yes.
Where can we contact you?
You can mail us at nitish.campusx@gmail.com
Payment/Registration related questions
Where do we have to make our payments? Your YouTube channel or website?
You have to make all your monthly payments on our website.
Can we pay the entire amount of Rs 5600 all at once?
Unfortunately no, the program follows a monthly subscription model.
What is the validity of monthly subscription? Suppose if I pay on 15th Jan, then do I have to pay again on 1st Feb or 15th Feb
15th Feb. The validity period is 30 days from the day you make the payment. So essentially you can join anytime you don’t have to wait for a month to end.
What if I don’t like the course after making the payment. What is the refund policy?
You get a 7 days refund period from the day you have made the payment.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmail.com
Post registration queries
Till when can I view the paid videos on the website?
This one is tricky, so read carefully. You can watch the videos till your subscription is valid. Suppose you have purchased subscription on 21st Jan, you will be able to watch all the past paid sessions in the period of 21st Jan to 20th Feb. But after 21st Feb you will have to purchase the subscription again.
But once the course is over and you have paid us Rs 5600(or 7 installments of Rs 799) you will be able to watch the paid sessions till Aug 2024.
Why lifetime validity is not provided?
Because of the low course fee.
Where can I reach out in case of a doubt after the session?
You will have to fill a google form provided in your dashboard and our team will contact you for a 1 on 1 doubt clearance session
If I join the program late, can I still ask past week doubts?
Yes, just select past week doubt in the doubt clearance google form.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmai.com
Certificate and Placement Assistance related queries
What is the criteria to get the certificate?
There are 2 criterias:
You have to pay the entire fee of Rs 5600
You have to attempt all the course assessments.
I am joining late. How can I pay payment of the earlier months?
You will get a link to pay fee of earlier months in your dashboard once you pay for the current month.
I have read that Placement assistance is a part of this program. What comes under Placement assistance?
This is to clarify that Placement assistance does not mean Placement guarantee. So we dont guarantee you any jobs or for that matter even interview calls. So if you are planning to join this course just for placements, I am afraid you will be disappointed. Here is what comes under placement assistance
Portfolio Building sessions
Soft skill sessions
Sessions with industry mentors
Discussion on Job hunting strategies
"""

In [3]:
# Convert Text into Sequence Number
tokenizer = Tokenizer()

In [4]:
#Tokenize on single document
tokenizer.fit_on_texts([faqs])

In [5]:
# Size of vocabulary build by tokenizer
len(tokenizer.word_index)

265

In [6]:
# This code creates sequences of increasing length from the tokenized sentences
input_sequences = []
for sentence in faqs.split('\n'):
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

  for i in range(1,len(tokenized_sentence)):
    input_sequences.append(tokenized_sentence[:i+1])

In [37]:
#print(input_sequences)

In [8]:
# find the maximum sequence
max_len = max([len(x) for x in input_sequences])

In [9]:
# Padd all squeces is same squences
padded_input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding='pre')

In [10]:
padded_input_sequences

array([[  0,   0,   0, ...,   0,  88,   1],
       [  0,   0,   0, ...,  88,   1,  13],
       [  0,   0,   0, ...,   0,  10,   8],
       ...,
       [  0,   0,   0, ..., 262,  18, 263],
       [  0,   0,   0, ...,  18, 263, 264],
       [  0,   0,   0, ..., 263, 264, 265]], dtype=int32)

In [11]:
# Slicing select all rows and columns except last
X = padded_input_sequences[:,:-1]

In [12]:
# Extract the target labels from the padded input sequences
y = padded_input_sequences[:,-1]

In [13]:
X.shape

(811, 56)

In [14]:
y.shape

(811,)

In [15]:
# one - hot- encoded the last columns
y = to_categorical(y,num_classes=283)

In [16]:
y.shape

(811, 283)

In [21]:
model = Sequential()
model.add(Embedding(input_dim=283, output_dim=100, input_length=56)) # input token to dense vector
model.add(LSTM(150, return_sequences=True))  # Important fix here
model.add(LSTM(150))
model.add(Dense(283, activation='softmax'))

model.build(input_shape=(None, 56))  # Build the model before summary



In [22]:
# prepare for training multi-classification problem
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 56, 100)        │        28,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 56, 150)        │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 150)            │       180,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 283)            │        42,733 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 402,233 (1.53 MB)

 Trainable params: 402,233 (1.53 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
# Train the model
model.fit(X,y,epochs=100)

Epoch 1/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 7s 164ms/step - accuracy: 0.0667 - loss: 5.5147
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 117ms/step - accuracy: 0.0749 - loss: 5.0770
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 116ms/step - accuracy: 0.0826 - loss: 5.0149
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 142ms/step - accuracy: 0.0822 - loss: 4.9155
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 116ms/step - accuracy: 0.0938 - loss: 4.8860
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 120ms/step - accuracy: 0.0833 - loss: 4.9251
Epoch 7/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 116ms/step - accuracy: 0.0898 - loss: 4.8918
Epoch 8/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 119ms/step - accuracy: 0.0747 - loss: 4.8659
Epoch 9/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 6s 142ms/step - accuracy: 0.0787 - loss: 4.8710
Epoch 10/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 115ms/step - accuracy: 0.0858 - loss: 4.7280
Epoch 11/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 113ms/step - accuracy: 0.0881 - loss: 4.6402
Epoch 12/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 5

In [25]:
import time
text = "what is the fee"

for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=56, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step
what is the fee of
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
what is the fee of monthly
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
what is the fee of monthly subscription
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
what is the fee of monthly subscription suppose
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
what is the fee of monthly subscription suppose if
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
what is the fee of monthly subscription suppose if i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
what is the fee of monthly subscription suppose if i pay
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
what is the fee of monthly subscription suppose if i pay on
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
what is the fee of monthly subscription suppose if i pay on 15th
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
what is the fee of monthly subscription suppose if i pay on 15th jan


In [34]:
# Make predictions
predictions = model.predict(X)

# Convert predictions to class labels (for classification tasks)
predicted_labels = np.argmax(predictions, axis=1)

# Convert true labels to class labels (if they are one-hot encoded)
true_labels = np.argmax(y, axis=1)

# Calculate accuracy manually
accuracy = np.mean(predicted_labels == true_labels)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step
Model Accuracy: 95.31%


In [ ]:
#### END #######